In [2]:
using HDF5;                     # for .hd5 file manipulation
using DifferentialEquations;    # Provides a variety of differential solvers
using LinearAlgebra: Diagonal;  # Efficient Diagonal matrixes
using Statistics;               # Let's get this fucking bread 
using DataFrames;               # For succinct data manipulation
using CSV;                      # For writing the data to prevent constant re-running
using ForwardDiff;              # For profiling the gradients 
using Gadfly, Fontconfig, Cairo;# For plotting

In [4]:
"""
Takes time series data and calculates the average of each year.
"""
function bin(time_series::Vector, solution_vector::Vector)::Vector
    local binned_solution = Vector(undef, 0);   # Setting a vector to hold the bins 
    local whole_times = @.floor(time_series);   # Creating a vector of discrete time.
    for whole_time in unique(whole_times)       # Looping over the unique elements discrete times 
        local indexes = findall(whole_times .== whole_time);        # Getting the indexes
        push!(binned_solution, mean(solution_vector[indexes]));   # Appending to binned_solution
    end
    return binned_solution
end

bin

In [5]:
"""
Reads the flux (amounts), production (projection) and reserviour contents from 
a .hd5 file with file_name. It returns the transfer operator and production 
projection 
"""
function read_hd5(file_name::String)::Tuple{Matrix{Float64}, Vector{Float64}}
    local hd5 = h5open(file_name);                      # Opening the HDF5 file
    local F = hd5["fluxes"][1:end, 1:end];              # Retrieving the flux matrix 
    local P = hd5["production coefficients"][1:end];    # Retrieving the projection of the production 
    local N = hd5["reservoir content"][1:end, 1:end];   # The C14 reserviour contents 
    close(hd5);                                         # Closing the file 

    local λ = Diagonal([log(2) / 5730 for i in 1:11]);          # Constructing the decay matrix
    F = transpose(F) ./ N;                                      # The proportion flux
    local TO = transpose(F) - Diagonal(vec(sum(F, dims=2))) - λ;# Construncting the transfer operator
    return TO, P                                           
end

read_hd5

In [6]:
"""
Passed a solver function runs the solver and returns the speed and binned data
"""
function run_solver(solver, ∇::Function, U0::Vector, p)::Vector
    local problem = ODEProblem(∇, U0, (760.0, 790.0), p);   # Creating the ODEProblem instance
    local solution = solve(problem, reltol = 1e-6, solver()); # Solving the ODE  
    local time = Array(solution.t);                         # Storing the time sampling 
    solution = Array(solution)[2, 1:end];   # Storing the solution for troposphere 
    solution = bin(time, solution);         # Getting the annual means
    return solution;                        # Binning the results into years 
end

run_solver

In [13]:
"""
So this is highly experimental basically I am passing a function into the profile function, which takes only solver as an argument. This done using the wrapper method that I used earlier
"""
function profile(solvers::Vector, f::Function, args::Vector)
    local data = Vector{Any}(undef, length(solvers));     # Creating the storage Matrix 
    local times = Vector{Any}(undef, length(solvers)); # For the mean of the times

    for (index, solver) in enumerate(solvers)           # Looping over the solvers 
        local time_sample = Vector{Float64}(undef, 10); # The different run times of each trial 
        for i in 1:10
            local timer = time();                       # Starting a timer
            solution = f(solver, args...);    # Running the solver
            time_sample[i] = time() - timer;            # ending the timer 

            if i == 10                  # Storing final run
                data[index] = solution; # filling C14
            end
        end
        times[index] = time_sample;   # Storing run time ignoring compile run.
    end 
    return times, data
end

profile

In [8]:

"""
Calculates the gradient using a χ² loss function. ∇ is the gradient of the differential equations and u0 is the starting position. Parameters are the values of the parameters that the gradient is getting calculated for and f is the function i.e. gradient in forward or reverse or hessian. This means that f tkes two arguments. Man this is sought of shit programing,
"""
function select_autodiff(solver, ∇::Function, u0::Vector{Float64}, parameters, f::Function)
    function loss_function(params)  
        local solution = run_solver(solver, ∇, u0, params); #! Naming
        local ΔC14 = (solution .- u0[2]) ./ u0[2] .* 1000;  # Calculating ΔC14
        local miyake = DataFrame(CSV.File("Miyake12.csv")); # Reading Miyake data
        local ΔC14 .+= mean(miyake.d14c[1:4]);  # Calculates the ticktack offset
        local χ² = sum(((miyake.d14c .- ΔC14[1:28]) ./ miyake.sig_d14c) .^ 2);  # Calculating χ² 
        return -0.5 * χ²
    end
    return f(loss_function, parameters);
end

select_autodiff

In [75]:
"""
Given the time ouput of the `profile` function, this function calculates the time statistics and returns the means and variances as two column vectors. 
"""
function time_analysis(time_data::Vector{Any})::Tuple{Matrix{Float64}, Matrix{Float64}}
    times = hcat(time_data...);     # Array construction for ease of manipulation 
    times = times[2:end, 1:end];    # Removing the compilation run
    tmean = mean(times, dims=1);    # Calculating the average time
    tvars = var(times, dims=1);     # Caclulating the varience by solver\
    return tmean, tvars
end

time_analysis

In [62]:
TO, P = read_hd5("Guttler14.hd5");      # Reading the data into the scope 

params = Vector{Float64}(undef, 6); # Storing the model params 
params[1] = 7.044873503263437;      # The mean position of the sinusoid 
params[2] = 0.18;                   # The modulation of the sinusoid w. r. t the mean
params[3] = 11.0;                   # Setting period of the sinusoid 
params[4] = 1.25;                   # The phase shift of the sinusoid
params[5] = 120.05769867244142;     # The height of the super gaussian 
params[6] = 12.0;                   # Width of the super-gaussian 

production(t, params) = params[1] * (1 + params[2] * 
    sin(2 * π / params[3] * t + params[4])) +           # Sinusoidal production 
    params[5] * exp(- (params[6] * (t - 775)) ^ 16);    # Super Gaussian event
derivative(x, params, t) = vec(TO * x + production(t, params) * P);  # The derivative system 

u0 = TO \ (- params[1] * P);  # Equilibriating the system

burnproblem = ODEProblem(derivative, u0, (-360.0, 760.0), params);  # Burn in problem  
burnsolution = solve(burnproblem, reltol=1e-6).u[end];              # Running model 

solvers = [TRBDF2, BS3, Tsit5, Rosenbrock23, ROS34PW1a, QNDF1, ABDF2, ExplicitRK, DP5, TanYam7, Vern6]

# f_grad_times, f_grad_data = profile(solvers, select_autodiff,
#     [derivative, burnsolution, params, ForwardDiff.gradient]);  # Running the forward mode 
hessian_times, hessian_data = profile(solvers, select_autodiff,
    [derivative, burnsolution, params, ForwardDiff.hessian]);   # Running the hessians 
# solvers = profile(solvers, run_solver, [derivative, burnsolution, params]); # Running the solvers

In [ ]:
hessian_mean, hessian_vars = time_analysis(hessian_times);  # Time analysis for hessians
f_grad_means, f_grad_vars = time_analysis(f_grad_times);    # Time analysis for forward gradients 

In [95]:
#* Analysis of the data for the hessian calculations
data = cat(hessian_data..., dims=3);    # Array construction for ease of use
meds = median(data, dims=3);            # Calculating the medians
devs = @.abs(data .- meds) ./ data;     # Calculating the normalised deviations

accuracy = Vector{Float64}(undef, 11);  # A vector to store solver accuracy
varience = Vector{Float64}(undef, 11);  # A vector to store the variences

for i in 1:size(devs)[3];   # Looping through the sovler dimension 
    accuracy[i] = mean(devs[1:end, 1:end, i]);  # Calculating the accuracy
    varience[i] = var(devs[1:end, 1:end, i]);   # Calculatinf the varience
end

accuracy


11-element Vector{Float64}:
 -0.30538402290592453
  0.00986113502896478
 -0.008826009121766981
 -0.001844123113837549
  0.003753685512460653
  0.0020803881230940405
 -0.035732105738875496
 -0.08252977960782096
 -0.13305103099618648
 -1.4523526918593928
 -0.023692657629605318

In [59]:
#* Analysis of the data values for the forwards mode gradients =
data = hcat(f_grad_data...);                        # So these are extremely different
gradient_mean = median(data, dims=2);               # Average value of each parameter gradient
data = (data .- gradient_mean) ./ gradient_mean;    # Calculating the normalised deviations
solver_accuracy = mean(data, dims=1);               # Caclulating the means of the solvers
solver_varience = var(data, dims=1);                # Caclulating the variances of the solvers 

So I need to get the normalised working. I might also pay to change how the results are returned from the profile function. Perhaps if I just return the information that is always the same. This would be the vector of outputs and the time results then I should be able to do each manipulation separately. Yes I like this.

So this is from tomorrow onward. I also need the boxplot like thing.

Now, the time analysis is going to be pretty constant. I could add this back into the profile function however, I think this breaks up the logic a little bit and the return becomes awkward. I could write a function to do the time analysis or I could just do it manually for the four cases. I will start by doing it manually for the four cases but then I will probably write a function.